In [49]:
import numpy as np 
import pandas as pd
import pulp

In [ ]:
"""
Parameters:
safety stock, maximum warehouse capacity
h: unit holding cost
max_p: production max per month
min_p = production min per month
wc = max inventory capacity per month
ss = min inventory capacity per month
I_0: initial_inventory
c_t: unit production cost in month t
d_t: demand of month t

Variables:
P_t: Amount produced in month t
I_t: Inventory at the end of period t

Constraints:
Inventory Constraints: I_{t-1} + X_t - d_t = I_t (Beginning Inventory)
                    I_t + P_t - d_t >= ss
                    I_t + P_t - d_t <= wc
Capacity Constraints: P_t <= max_p
                      P_t >= min_p

Objective: Min Sum(((2*I_t)+P_t - d_t).5*h*c_t + c_t*P_t)


"""

In [50]:
filepath = '/Volumes/Elements/khartless/Projects/Data/'
parameters_name = 'Upton.xlsx'

input_data = pd.read_excel(filepath+parameters_name, sheet_name= 'Production', index_col=[0])


In [51]:
initial_inventory = 2750
safety_stock = 1500
max_capacity = 6000
carrying_cost = .015

In [52]:
model = pulp.LpProblem(name='prod_planning_upton', sense=pulp.LpMinimize)

In [53]:
production_variables = pulp.LpVariable.dicts(name='P', indexs=input_data.index,
                                             lowBound=0, cat=pulp.LpContinuous)

inventory_variables = pulp.LpVariable.dicts(name='I', indexs=input_data.index,
                                            lowBound=0, cat=pulp.LpContinuous)

In [54]:
for period, value in input_data.iloc[1:].iterrows():
    model.addConstraint(pulp.LpConstraint(
        e=inventory_variables[period - 1] + production_variables[period] - inventory_variables[period],
        sense=pulp.LpConstraintEQ,
        name='inv_balance' + str(period),
        rhs=value.Demand))

In [55]:
# inv balance for first period
model.addConstraint(pulp.LpConstraint(
    e=production_variables[1] - inventory_variables[2],
    sense=pulp.LpConstraintEQ,
    name='inv_balance1',
    rhs=input_data.iloc[1].Demand - initial_inventory))

In [56]:
for index, value in production_variables.items():
    model.addConstraint(pulp.LpConstraint(
        e=value,
        sense=pulp.LpConstraintLE,
        name='prod_cap_month_' + str(index),
        rhs=input_data.loc[index].Max_Production))

In [57]:
for index, value in production_variables.items():
    model.addConstraint(pulp.LpConstraint(
        e=value,
        sense=pulp.LpConstraintGE,
        name='prod_min_month_' + str(index),
        rhs=input_data.loc[index].Min_Production))


In [60]:
for index, value in inventory_variables.items():
    model.addConstraint(pulp.LpConstraint(
        e=value,
        sense=pulp.LpConstraintLE,
        name='inventory_cap_month_' + str(index),
        rhs=max_capacity))

for index, value in inventory_variables.items():
    model.addConstraint(pulp.LpConstraint(
        e=value,
        sense=pulp.LpConstraintGE,
        name='inventory_min_month_' + str(index),
        rhs=safety_stock))

In [68]:
#Objective: Min Sum(((2*I_t)+P_t - d_t).5*h*c_t + c_t*P_t)

total_holding_cost = pulp.lpSum(((2*inventory_variables[index])+production_variables[index] - row['Demand'])*.5 * carrying_cost * row['Unit_Production_Cost']
                                for index, row in input_data.iterrows())
total_production_cost = pulp.lpSum(row['Unit_Production_Cost'] * production_variables[index]
                                   for index, row in input_data.iterrows())

objective = total_holding_cost + total_production_cost

model.setObjective(objective)

In [73]:
pulp.value(model.objective)

6446184.375